In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from transformers import ViTModel, ViTFeatureExtractor

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        features = model(**inputs).last_hidden_state[:, 0, :]

    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...")])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [22:58<00:00,  8.27it/s]

Extracted feature shape: (11399, 768)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 32), alpha=1e-3, max_iter=300, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 63.64% | Prec: 70.02% | Rec: 63.64% | F1: 64.85% | Bal Acc: 55.76% | MCC: 0.6190
LogisticRegression        | Acc: 91.18% | Prec: 88.88% | Rec: 91.18% | F1: 89.54% | Bal Acc: 61.42% | MCC: 0.9057
SVMLinear                 | Acc: 92.98% | Prec: 91.59% | Rec: 92.98% | F1: 91.91% | Bal Acc: 71.19% | MCC: 0.9250
SVMPolynomial             | Acc: 88.38% | Prec: 84.99% | Rec: 88.38% | F1: 85.94% | Bal Acc: 52.69% | MCC: 0.8755
SVMRadialBasis            | Acc: 87.11% | Prec: 82.99% | Rec: 87.11% | F1: 84.09% | Bal Acc: 48.77% | MCC: 0.8619
NearestNeighbor           | Acc: 77.15% | Prec: 74.98% | Rec: 77.15% | F1: 74.81% | Bal Acc: 46.03% | MCC: 0.7547
RandomForest              | Acc: 73.64% | Prec: 69.30% | Rec: 73.64% | F1: 68.99% | Bal Acc: 30.75% | MCC: 0.7168
MLP_Baseline              | Acc: 93.20% | Prec: 92.43% | Rec: 93.20% | F1: 92.49% | Bal Acc: 74.46% | MCC: 0.9273


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Cymindis cribricollis,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Synuchus impunctatus,Synuchus impunctatus,Agonum punctiforme,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/17.ViT-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/17.ViT-linear-probing-species-metrics.csv", index=False)